In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [3]:
train_df = pd.read_csv("train.csv", delimiter=',')
train_df

,Unnamed: 0,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,0,40.108910,-83.092860,8336,0,3,-135060.089443,86013.396489,1206.094242,52287.082257,0
1,1,39.865420,-84.062800,18403,1,0,-1766.845055,14985.640180,477.494992,168836.215743,1
2,2,39.102660,-84.524680,14022,0,3,-177302.873693,44881.958005,1463.339889,130388.243325,0
3,3,39.101480,-84.523410,11051,0,0,209049.997460,0.000000,95.340075,103267.727546,1
4,4,41.062130,-81.537840,3243,0,3,8669.269507,0.000000,399.421926,177532.206618,1
...,...,...,...,...,...,...,...,...,...,...,...
159996,159996,41.567510,-72.727300,11307,0,3,16243.910806,31628.125379,425.822833,171803.513497,1
159997,159997,41.857517,-88.143778,6324,0,3,28018.854840,48674.176247,401.849141,68368.711459,1
159998,159998,35.261260,-80.837000,15370,1,1,101012.486516,0.000000,199.255523,190942.481245,0
159999,159999,35.261260,-80.836997,9384,0,1,6461.647033,0.000000,507.408851,18122.078134,1


In [4]:
train_df.loc[train_df["company"].value_counts()[train_df["company"]].values <= 10, "company"] = -1
le = preprocessing.LabelEncoder()
le.fit(train_df["company"])
train_df["enc_company"] = le.transform(train_df["company"])


geo_cluster = KMeans(n_clusters=300, random_state=0)
geo_cluster.fit(train_df[['longitude', 'latitude']])

train_df["cluster"] = geo_cluster.predict(train_df[['longitude', 'latitude']])


train_df[["n_fin_1", "n_fin_2", "n_fin_3", "n_fin_4"]] =preprocessing.normalize(train_df[["fin_1", "fin_2", "fin_3", "fin_4"]], norm='l2')

In [6]:
x_train, x_validation, y_train, y_validation = train_test_split(train_df.drop('target', axis=1), 
                                                      train_df.target, test_size=0.1, random_state=1)

In [7]:
train_df['company'].nunique()

1385

In [8]:
test_df = pd.read_csv("test.csv", delimiter=',')
test_df

,Unnamed: 0,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,160000,39.284310,-76.735350,18187,1,1,113033.389907,0.000000,270.906219,31222.780176,0
1,160001,36.758509,-76.344861,11208,0,0,-87239.590275,73759.387510,759.194862,237587.544996,1
2,160002,43.402802,-75.217100,7437,1,4,-122084.498620,15528.109943,1039.655934,29612.346982,0
3,160003,41.871160,-87.848570,17362,0,1,87355.127256,0.000000,214.594205,163526.475818,1
4,160004,42.161296,-88.129184,11515,0,3,33014.437946,0.000000,379.819724,67499.397999,1
...,...,...,...,...,...,...,...,...,...,...,...
39995,199995,35.241600,-80.983740,11640,0,3,194648.093228,0.000000,95.701485,37584.380280,1
39996,199996,35.241730,-80.983750,11849,0,3,-53202.007008,9641.082811,653.516413,189604.463353,0
39997,199997,35.290596,-80.756953,3987,0,3,-102398.568309,47105.929570,863.135873,26231.880089,0
39998,199998,35.204460,-80.720190,12287,0,0,32066.188506,6659.705140,363.861756,116288.412218,1


In [9]:
test_df.loc[test_df["company"].value_counts()[test_df["company"]].values <= 10, "company"] = -1
test_df["enc_company"] = le.transform(test_df["company"])

test_df["cluster"] = geo_cluster.predict(test_df[['longitude', 'latitude']])


test_df[["n_fin_1", "n_fin_2", "n_fin_3", "n_fin_4"]] =preprocessing.normalize(test_df[["fin_1", "fin_2", "fin_3", "fin_4"]], norm='l2')

In [17]:
input_col = keras.Input(shape=(5,), name="input_col")  


x1 = layers.Dense(32)(input_col)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dense(32)(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Reshape((1, 32))(x1)


company_input = keras.Input(shape=(1,), name="company_input") 
num_companies=le.classes_.shape[0]
x2 = layers.Embedding(num_companies, 128)(company_input)
x2 = layers.Dense(32)(x2)


type_input = keras.Input(shape=(1,), name="type_input") 
num_type = 5
x3 = layers.Embedding(num_type, 16)(type_input)
x3 = layers.Dense(16)(x3)


geo_input = keras.Input(shape=(1,), name="geo_input") 
num_clusters = 300
x4 = layers.Embedding(num_clusters, 128)(geo_input)
x4 = layers.Dense(32)(x4)


x = layers.Concatenate()([x1, x2, x3, x4])
x = layers.Dense(64)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(64)(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(1)(x)


model = keras.Model(
    inputs=[input_col, company_input, type_input,geo_input ],
    outputs=outputs,
)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [14]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="./checkpoint4",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


history = model.fit({"input_col": x_train[["n_fin_1", "n_fin_2", "n_fin_3", "n_fin_4", "is_local"]],
                     "company_input": x_train["enc_company"], "type_input": x_train["type"], "geo_input": x_train["cluster"]},
                    y_train, batch_size=128, epochs=10, 
                    validation_data=({"input_col": x_validation[["n_fin_1", "n_fin_2", "n_fin_3", "n_fin_4", "is_local"]],
                     "company_input": x_validation["enc_company"], "type_input": x_validation["type"],
                    "geo_input": x_validation["cluster"]},
                    y_validation), callbacks=[model_checkpoint_callback])

Epoch 1/10
1125/1125 [==============================] - 12s 8ms/step - loss: 3.1926 - accuracy: 0.7356 - val_loss: 2.9508 - val_accuracy: 0.7646
Epoch 2/10
1125/1125 [==============================] - 8s 7ms/step - loss: 3.0670 - accuracy: 0.7280 - val_loss: 6.5711 - val_accuracy: 0.5410
Epoch 3/10
1125/1125 [==============================] - 8s 7ms/step - loss: 3.8865 - accuracy: 0.7139 - val_loss: 3.0419 - val_accuracy: 0.7690
Epoch 4/10
1125/1125 [==============================] - 8s 7ms/step - loss: 3.5015 - accuracy: 0.7300 - val_loss: 4.1930 - val_accuracy: 0.6942
Epoch 5/10
1125/1125 [==============================] - 8s 7ms/step - loss: 3.7381 - accuracy: 0.7219 - val_loss: 3.3427 - val_accuracy: 0.7452
Epoch 6/10
1125/1125 [==============================] - 8s 7ms/step - loss: 3.5420 - accuracy: 0.7318 - val_loss: 3.7470 - val_accuracy: 0.7203
Epoch 7/10
1125/1125 [==============================] - 8s 7ms/step - loss: 3.5776 - accuracy: 0.7249 - val_loss: 3.2644 - val_accuracy

In [15]:
print(model.summary())
test_scores = model.evaluate({"input_col": test_df[["n_fin_1", "n_fin_2", "n_fin_3", "n_fin_4", "is_local"]],
                     "company_input": test_df["enc_company"], "type_input": test_df["type"], 
                              "geo_input": test_df["cluster"]}, test_df["target"], verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_col (InputLayer)         [(None, 5)]          0           []                               
                                                                                                  
 dense_8 (Dense)                (None, 32)           192         ['input_col[0][0]']              
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 32)          128         ['dense_8[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 dense_9 (Dense)                (None, 32)           1056        ['batch_normalization_4[0][